In [ ]:
def extract_legal_references(text):
    # Kết quả danh sách các tham chiếu
    result = []
    
    # Xác định số Điều hiện tại (nếu có)
    current_articles = []
    
    # Xác định số Khoản hiện tại (nếu có)
    current_clauses = []
    
    # Tìm và xử lý các mẫu có trong văn bản
    # Điều (Articles)
    article_matches = re.findall(r"điều (\d+(?:, \d+)*(?: và \d+)?)", text, re.IGNORECASE)
    if article_matches:
        for match in article_matches:
            articles = re.split(r", | và ", match)
            current_articles = [art.strip() for art in articles]
            result.extend([f"{art}.0.0" for art in current_articles])
    
    # Khoản (Clauses) và Điểm (Points)
    clause_matches = re.finditer(r"(khoản (\d+(?:, \d+)*(?: và \d+)?))(.*?điều (\d+|này))?", text, re.IGNORECASE)
    for clause_match in clause_matches:
        clause_numbers = clause_match.group(2)
        clause_articles = clause_match.group(4)
        
        # Xác định các khoản
        clauses = re.split(r", | và ", clause_numbers)
        current_clauses = [cl.strip() for cl in clauses]
        
        # Xác định điều liên quan nếu có
        if clause_articles and clause_articles.lower() != "này":
            current_articles = [clause_articles.strip()]
        
        # Điểm (Points)
        point_section = clause_match.group(3)
        if point_section:
            point_matches = re.findall(r"điểm ([a-z](?:, [a-z])*(?: và [a-z])?)", point_section, re.IGNORECASE)
            for point_match in point_matches:
                points = re.split(r", | và ", point_match)
                for article in current_articles:
                    for clause in current_clauses:
                        result.extend([f"{article}.{clause}.{pt.strip()}" for pt in points])
        else:
            # Nếu không có Điểm, chỉ thêm Khoản và Điều
            for article in current_articles:
                result.extend([f"{article}.{clause}.0" for clause in current_clauses])
    
    # Loại bỏ trùng lặp
    return sorted(set(result))

def extract_legal_references_other_way(text):
    # Kết quả danh sách các tham chiếu
    result = []
    
    # Xác định Điều và Khoản hiện tại (nếu có)
    current_articles = ["0"]
    current_clauses = ["0"]
    
    # Xử lý các mẫu trong văn bản
    # Điều (Articles)
    article_matches = re.findall(r"điều (\d+(?:, \d+)*(?: và \d+)?)", text, re.IGNORECASE)
    if article_matches:
        for match in article_matches:
            articles = re.split(r", | và ", match)
            current_articles = [art.strip() for art in articles]
            result.extend([f"{art}.0.0" for art in current_articles])
    
    # Khoản (Clauses) và Điểm (Points)
    clause_matches = re.finditer(r"(khoản (\d+(?:, \d+)*(?: và \d+)?|này))(.*?điều (\d+|này))?", text, re.IGNORECASE)
    for clause_match in clause_matches:
        clause_numbers = clause_match.group(2)
        clause_articles = clause_match.group(4)
        
        # Xác định các khoản
        if clause_numbers.lower() == "này":
            clauses = current_clauses
        else:
            clauses = re.split(r", | và ", clause_numbers)
            current_clauses = [cl.strip() for cl in clauses]
        
        # Xác định điều liên quan nếu có
        if clause_articles and clause_articles.lower() != "này":
            current_articles = [clause_articles.strip()]
        elif clause_articles and clause_articles.lower() == "này":
            # "Điều này" giữ nguyên điều hiện tại
            pass
        
        # Điểm (Points)
        point_section = clause_match.group(3)
        if point_section:
            point_matches = re.findall(r"điểm ([a-z](?:, [a-z])*(?: và [a-z])?)", point_section, re.IGNORECASE)
            for point_match in point_matches:
                points = re.split(r", | và ", point_match)
                for article in current_articles:
                    for clause in clauses:
                        result.extend([f"{article}.{clause}.{pt.strip()}" for pt in points])
        else:
            # Nếu không có Điểm, chỉ thêm Khoản và Điều
            for article in current_articles:
                result.extend([f"{article}.{clause}.0" for clause in clauses])
    
    # Xử lý các tham chiếu như "Điều này" hoặc "Khoản này"
    result = [ref.replace(".này", ".0").replace("này", "0") for ref in result]
    
    # Loại bỏ trùng lặp và sắp xếp
    return sorted(set(result))